# INFO 2950 Final Project - Phase IV Data

## Data Descriptions:

### What are the observations (rows) and the attributes (columns)?
- **Observations**:  
  Each row in the dataset represents a specific entity (e.g., a year-state combination for state-level EV registrations or federal incentives).  
- **Attributes**:  
  - **Year**  
  - **State**  
  - **Total EV registrations**
  - **Total EV registrations per Capita**
  - **Lowest EV Price of the Year**
  - **Population (by State by Year)**
  - **Federal Incentives Count (by year)**

---

### Why was this dataset created?
The dataset was created to explore and analyze the factors influencing electric vehicle (EV) adoption across states, focusing on variables such as federal incentives, pricing, renewable energy production, and infrastructure availability. This analysis aims to provide insights into how policy and economic factors shape EV adoption trends.

---

### Who funded the creation of the dataset?
The dataset compilation and underlying sources are funded by public organizations such as:
- U.S. Department of Energy (DOE)
- U.S. Energy Information Administration (EIA)
- Open Charge Map
- U.S. Census Bureau 

---

### What processes might have influenced what data was observed and recorded, and what was not?
- Data accuracy may vary depending on reporting standards and processes across states.  
- External events, such as changes in data collection methods or interruptions (e.g., pandemic years), may influence what was observed or omitted.  

---

### What preprocessing was done, and how did the data come to be in the form that you are using?
- Converted EV registration data into numeric formats.  
- Analyzed and selected the top 5 states in EV growth and registrations.  
- Filtered the EV registration data for the selected 5 states: `California`, `Florida`, `Texas`, `Washington`, `New Jersey`.  
- Converted EV Manufacturer Suggested Retail Prices (MSRP) data into numeric formats and dropped NaN values.  
- Calculated the lowest EV price of the year.
- Merged the filtered registration data with the lowest prices by `Year`.
- Removed unnecessary columns (Law Id, Text, Agency, Topic, etc.) from incentives dataset.
- Converted enactment and expiration dates to datetime format and filtered for US-level incentives only. 
- Manually added missing enactment dates for specific policies.
- Created a yearly count of active federal incentives.
- Combined the cleaned federal incentives data with the merged dataset of EV registrations and lowest prices using `Year` and `State` as the common key.

---

### If people are involved, were they aware of the data collection, and if so, what purpose did they expect the data to be used for?
- As the data primarily originates from governmental agencies and public resources, individuals contributing to or represented in the data (e.g., EV owners) may not have been directly aware of its collection.  
- Agencies and organizations were likely aware that the data would be used for research, policy evaluation, and public awareness campaigns.

---

### Where can your raw source data be found?  
The raw source data is hosted online and can be accessed at the following links:  

1. **Vehicle Registration Counts by State**  
   - **Data Source**: U.S. Department of Energy - Alternative Fuels Data Center (AFDC)  
   - **URL**: [Vehicle Registration Data](https://afdc.energy.gov/vehicle-registration?year=2023)  

2. **Renewable and Total Energy Production by State**  
   - **Data Source**: U.S. Energy Information Administration (EIA) – State Energy Data System (SEDS)  
   - **URLs**:  
     - [Website](https://www.eia.gov/renewable/data.php)  
     - [Production Report PDF](https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf) (pages 18-119)  

3. **EV Retail Prices by Models**  
   - **Data Source**: U.S. Department of Energy - Office of Energy Efficiency & Renewable Energy  
   - **URL**: [EV Pricing Data](https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=0)  

4. **EV Charge Stations**  
   - **Data Source**: Open Charge Map  
   - **URL**: [EV Charging Data](https://openchargemap.org/site/develop/api#/)  

5. **Federal and State Incentives Related to Alternative Fuels and Vehicles**  
   - **Data Source**: U.S. Department of Energy - Alternative Fuels Data Center (AFDC)  
   - **URL**: [Federal and State Incentives Data](https://afdc.energy.gov/laws/state)
   - The specific CSV file can be downloaded by visiting the provided URL and selecting "Download Data" at the bottom of the page. Once redirected, apply the following filters for each field before downloading:
       - Dataset: "Laws and Incentives"
       - Jurisdiction: "All"
       - Technology: "All-Electric Vehicles (EV)"
       - Incentive Type: "All"
       - Regulation Type: "All"
       - User Type: "All"
       - File Format: "CSV (opens in Excel)"

6. **State Population Totals**  
   - **Data Source**: United States Census Bureau 
   - **URL**: [State Population Data (2010-2019)](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html#par_textimage_1574439295) and [State Population Data (2020-2023)](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html)
   - The files can be downloaded directly from their respective pages.
       - For population data from **2010** to **2019**, download the file titled:
           -  Annual Estimates of the Resident Population for the United States, Regions, States, and Puerto Rico: April 1, 2010 to July 1, 2019 (NST-EST2019-01) [<1.0 MB].
        - For population data from **2020** to **2023**, download the file titled:
            - Annual Estimates of the Resident Population for the United States, Regions, States, District of Columbia, and Puerto Rico: April 1, 2020 to July 1, 2023 (NST-EST2023-POP) [<1.0 MB].

## Importing:

In [ ]:
#pip install pdfplumber

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests
from bs4 import BeautifulSoup
import os
import re
import pdfplumber
import duckdb
from io import BytesIO

## Data Scraping:

1. **Vehicle Registration Counts by State**
   * <u>Data Source</u>: US Department of Energy - Alternative Fuels Data Center (AFDC)
   * <u>URL</u>: https://afdc.energy.gov/vehicle-registration?year=2023

In [ ]:
#defining url to scrape from
afdc_url = "https://afdc.energy.gov/vehicle-registration?year={}"

#defining year range to scrape for
years = range(2016, 2024)

compiled_data = []

#scrape data using requests
for year in years:
    url = afdc_url.format(year)
    afdc_result = requests.get(url)

    if afdc_result.status_code == 200:
        print(f"Scraping data for {year}...")
                
    page = BeautifulSoup(afdc_result.text, 'html.parser')
                
    table = page.find('table')

    #scrape rows of data from table
    if table:
        rows = table.find_all('tr')

        print(f"Found {len(rows)} rows in the table for {year}.")
                    
        for row in rows[2:]: 
            cols = row.find_all('td')
            cols = [col.text.strip() for col in cols]
                        
            compiled_data.append([year] + cols)

    else:
        print(f"Failed to retrieve data for {year}: {afdc_result.status_code} - {afdc_result.reason}")

#create table for data
if table:
        header_row = table.find('tbody').find_all('tr')[0]
        headers = [td['headers'] for td in header_row.find_all('td')]
        clean_headers = []
        for header in headers:
            if header[0].isupper():
                    clean_headers.append(header[0].strip())
            else:
                    clean_headers.append(header[0].strip().capitalize())
        
        print(f"Headers found: {clean_headers}")

compiled_df = pd.DataFrame(compiled_data, columns=["Year"] + clean_headers)

print(compiled_df.head(n=5))

2. **Renewable and Total Energy Production by State**
    * <u>Data Source</u>: US Energy Information Administration (EIA) – State Energy Data System (SEDS)
    * <u>URL(s)</u>: <br>
        Website - https://www.eia.gov/renewable/data.php <br>
        File - https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf (pg 18-119)

In [ ]:
def extract_pdf(pdf_url, start_page, end_page):
    response = requests.get(pdf_url)
    
    if response.status_code == 200:
        file = BytesIO(response.content)

        #use pdfplumber to open pdf and scrape from it
        with pdfplumber.open(file) as pdf:
            all_tables = []
            
            for i in range(start_page, end_page):
                one_page = []  
                count = 0  
                
                page = pdf.pages[i]
                text = page.extract_text()

                #look for data entries in table to extract text
                entries = re.findall(r'NA|\(s\)|\b\d{1,3}(?:,\d{3})*(?:\.\d+)?\b', text)

                #replace commas and with empty string
                cleaned_entries = [float(num.replace(',', ''))
                                   if num not in ['NA', '(s)']
                                   else num for num in entries]
                
                one_page.extend(cleaned_entries)
                count += len(cleaned_entries)

                one_page = one_page[:354]
                
                reshaped_data = np.array(one_page).reshape(59, 6)
                
                all_tables.append(reshaped_data)
                
            return all_tables  

    else:
        return "Something went wrong"

pdf_url = 'https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf'

data = extract_pdf(pdf_url, 17, 119)

In [ ]:
states = ["Alabama", "Alaska", "Arizona", "Arkansas", 
          "California", "Colorado", "Connecticut", 
          "Delaware", "District of Columbia", "Florida", 
          "Georgia", "Hawaii", "Idaho", "Illinois", 
          "Indiana", "Iowa", "Kansas", "Kentucky", 
          "Louisiana", "Maine", "Maryland", "Massachusetts", 
          "Michigan", "Minnesota", "Mississippi", "Missouri", 
          "Montana", "Nebraska", "Nevada", "New Hampshire", 
          "New Jersey", "New Mexico", "New York", "North Carolina", 
          "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", 
          "Rhode Island", "South Carolina", "South Dakota", "Tennessee",
          "Texas", "Utah", "Vermont", "Virginia", "Washington", 
          "West Virginia", "Wisconsin", "Wyoming"]

#create years range for years table column
years = [1960] + list(range(1965, 2023))
repeated_years = years * 51

#define Series for data appending
physical_units = []
thermal_units = []

for j in range(len(data)):
    df = pd.DataFrame(data[j])

    #append all data for physical units
    if j % 2 == 0: 
        state_index = j//2
        df.insert(0, 'State', states[state_index])
        physical_units.append(df)

    #append all data for thermal units
    else:
        state_index = (j-1)//2
        df.insert(0, 'State', states[state_index])
        thermal_units.append(df)


## In Physical Units
physical_units_df = pd.concat(physical_units, ignore_index=True)

physical_units_df.insert(1, 'Units', "Physical")
physical_units_df.rename(
    columns={
            0: 'Coal (K short tons)', 
            1: 'Natural Gas (M cubic ft)',
            2: 'Crude Oil (K barrels)',
            3: 'Fuel Ethanol (K barrels)',
            4: 'Biodiesel (K barrels)',
            5: 'Renewable Diesel (K barrels)'
        }, inplace=True)

#add year column to dataframe
physical_units_df.insert(0, 'Year', repeated_years)


## In Thermal Units
thermal_units_df = pd.concat(thermal_units, ignore_index=True)

thermal_units_df.insert(1, 'Units', "Thermal")
thermal_units_df.rename(
        columns={
            0: 'Coal (T Btu)', 
            1: 'Natural Gas (T Btu)',
            2: 'Crude Oil (T Btu)',
            3: 'Fuel Ethanol (T Btu)',
            4: 'Biodiesel (T Btu)',
            5: 'Renewable Diesel (T Btu)'
        }, inplace=True)

#add year column to dataframe
thermal_units_df.insert(0, 'Year', repeated_years)


display(physical_units_df)
display(thermal_units_df)

3. **EV Manufacturer Suggested Retail Prices and Annual Fuel Cost (by Models)**
    * <u>Data Source</u>: US Department of Energy - Office of Energy efficiency & Renewable Energy
    * <u>URL</u>: https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=0

In [ ]:
# List of URLs for multiple pages
urls = [
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=2',
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=2&tabView=2',
    'https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=3&tabView=2'
]

# Initialize lists for data storage
years = []
models = []
annual_fuel_costs = []
msrp_prices = []

# Loop over each URL to fetch data from multiple pages
for url in urls:
    # Send request and parse HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all 'tr' tags with the 'ymm-row' class
    vehicle_rows = soup.find_all('tr', class_='ymm-row')

    # Loop over each 'ymm-row' and find the next row for cost and msrp
    for vehicle_row in vehicle_rows:
        # Get vehicle details
        vehicle_tag = vehicle_row.find('a')
        if vehicle_tag:
            vehicle_text = vehicle_tag.get_text(strip=True)
            year, model = vehicle_text.split(' ', 1)  # Split on the first space to separate year and model
            years.append(year)
            models.append(model)
        else:
            years.append(np.nan)
            models.append(np.nan)

        # Find the next sibling row for cost and MSRP
        cost_row = vehicle_row.find_next_sibling('tr')
        if cost_row:
            fuel_cost_tag = cost_row.find('td', class_='ann-fuel-cost')
            msrp_tag = cost_row.find('td', class_='msrp')

            # Check if MSRP exists, else skip this entry
            if msrp_tag:
                # Process fuel cost
                if fuel_cost_tag:
                    fuel_cost_text = fuel_cost_tag.get_text(strip=True)
                    annual_fuel_costs.append(fuel_cost_text.replace('\r', '') \
                                             .replace('\n', '').replace('\t', ''))
                else:
                    annual_fuel_costs.append(np.nan)

                # Process MSRP and clean up
                msrp_text = msrp_tag.get_text(strip=True)
                msrp_cleaned = msrp_text.replace('\r', '').replace('\n', '').replace('\t', '')
                msrp_prices.append(msrp_cleaned)
            else:
                # If MSRP is missing, skip this entry
                years.pop()  # Remove the last added item from years
                models.pop()  # Remove the last added item from models
        else:
            # If there's no next row, remove the last entry
            years.pop()
            models.pop()

# Create DataFrame
pricing_data = {
    'Year': years,
    'Model': models,
    'Annual Fuel Cost': annual_fuel_costs,
    'MSRP': msrp_prices
}
pricing_df = pd.DataFrame(pricing_data)

# Convert 'Year' to numeric for sorting, ignoring errors for NaN
pricing_df['Year'] = pd.to_numeric(pricing_df['Year'], errors='coerce')

# Sort by Year
pricing_df_sorted = pricing_df.sort_values(by='Year', ascending=True)

# Display the DataFrame
print(pricing_df_sorted)

4. **EV Charge Stations**
    * <u>Data Source</u>: Open Charge Map
    * <u>Website URL</u>: https://openchargemap.org/site/develop/api#
    * <u>Data Description</u>: Open Charge Map's API provides access to a global database of electric vehicle charging locations. It offers a range of functionalities, including retrieving nearby charging stations, filtering stations by country, operator, or status, and contributing data such as new locations or updates. The API supports JSON and XML formats, allowing developers to integrate charging point information into apps, navigation systems, or research projects. The data is community-driven and free to use.

`ocm_us_data.csv` contains charging station data in the US. This csv file was generated by using Open Charge Map's public API. Using Postman, we sent the following HTTPS GET request (API key is ommitted):

`https://api.openchargemap.io/v3/poi?key=apikey&countrycode=US&includecomments=false&maxresults=9999999&output=csv`

The `key` parameter sets our API key so we can call the API. The `countrycode` parameter filters for US charging stations only. The `include comments` parameter removes use comments from the returned data for efficiency. The `maxresults` parameter ensures all data on US charging stations is returned (100 is default, maxresults=9999999 returns the same data as maxresults=999999 so we know our upper limit is high enough). Finally, the `output` parameter ensures our returned data is in the form of a csv file. 

## Data Cleaning - Cleaning Individual Variables:

1. **EV Registration Data**

Since we want to predict the EV registration rate, we first need to convert the registration data into numeric types for future use.

In [ ]:
def clean_regis(regis):
    #cleaning na entries
    if pd.isna(regis):
        return None
    #cleaning commas out from floats
    regis = regis.replace(',', '').strip()
    return float(regis)

compiled_df['Electric'] = compiled_df['Electric'].apply(clean_regis)
print(compiled_df)

**By Percent Growth Between 2016-2023**

In [ ]:
#creating new df that calculates percent change
change_df = duckdb.sql("""
    SELECT Year, State, Electric
    FROM compiled_df
    WHERE Year = 2016 OR Year = 2023
    """).df().pivot(
    index='State', 
    columns='Year', 
    values='Electric').reset_index()

change_df.rename(columns={2016: 'Old', 2023: 'New'}, inplace=True)

#creating new df that calculates percent growth
growth_df = duckdb.sql("""
    SELECT State, (New-Old)/Old*100 AS "% Growth"
    FROM change_df
    ORDER BY "% Growth" DESC
    """).df()

print(growth_df.head(n=5))

**By Registration Count in 2023**

In [ ]:
#ranking EV registration data
recent_df = duckdb.sql("""
    SELECT State, Electric AS "2023"
    FROM compiled_df
    WHERE Year = 2023
    ORDER BY "2023" DESC
    """).df()

us_total = recent_df.loc[recent_df['State'] == 'United States', '2023'].values[0]

#creating new column that shows EV percentage of US
recent_df['% in US'] = recent_df['2023'] / us_total * 100

display(recent_df.head(n=6))

**Top 5 States By Growth & Registration (2023)** 

In [ ]:
#filtering out United States and joining growth_df and recent_df for holistic summary
summary_df = duckdb.sql("""
    SELECT *
    FROM growth_df
    INNER JOIN recent_df
    USING (State)
    WHERE State != 'United States'
    """).df()

#sort by top growth
top_growth = summary_df.sort_values(by='% Growth', ascending=False)
top_count = summary_df.sort_values(by='2023', ascending=False)

#find top 5
display(top_growth.head(n=5))
display(top_count.head(n=5))

top_states = top_count.loc[0:4, 'State'].to_list()
print(top_states)

Given that the highest-registration states have meaningful growth rates while the opposite is not true, we decided to focus on the top 5 states based on total registration: California, Florida, Texas, Washington, New Jersey.

**Filter the Registration Data Frame for the Selected States** 

We filtered the registration data to include only the "Electric" column because our analysis focuses exclusively on electric vehicle (EV) registrations. Other vehicle types, such as PHEV or HEV, are outside the scope of this study and could introduce unnecessary noise into the dataset. By isolating EV registrations, we ensure that our findings are directly relevant to understanding the factors influencing EV adoption.

We also filtered the data to focus on the top 5 states in EV growth and registrations because these states represent the most significant trends in EV adoption. By analyzing these leading states, we can better understand the factors driving EV growth and adoption in regions where EV markets are most active, providing insights that are both impactful and scalable.

In [ ]:
#Filter the registration data for only "Electric" vehicles
ev_registration = compiled_df[['Year', 'State', 'Electric']]

#Filter the registration data to include only the selected states
selected_states = ['California', 'Florida', 'Texas', 'Washington', 'New Jersey']
registrations_data = ev_registration[ev_registration['State'].isin(selected_states)]
print(registrations_data.head())

**Get the Registration Data Per Capita for the Selected States** 

EV registrations per capita provide a standardized measure of EV adoption that accounts for population differences between states. This metric allows us to make fair comparisons of EV adoption levels across states with varying population sizes and identify trends that may not be apparent from total registration numbers alone.

To calculate EV registrations per capita, we started by loading state-level population data for 2010–2019 and 2020–2023. We cleaned the data by renaming columns, removing unnecessary ones like Census, and fixing state names by stripping leading dots. Then, we filtered the data to keep only the five selected states—California, Florida, Texas, Washington, and New Jersey—and removed commas from population values to convert them into numbers. Population data for 2016–2019 and 2020–2023 was combined into one dataset and reshaped into a long format with State, Year, and Population columns. This was merged with the EV registrations data using State and Year as keys. Finally, we calculated the EV registrations per capita by dividing the total EV registrations by the population for each state and year, adding the result as a new column to the dataset.

In [ ]:
# Load the population datasets 
# (U.S. Census Bureau, Population Division, 2019
# (U.S. Census Bureau, Population Division, 2023)
population_2010_2019 = pd.read_csv("csv_data_files/pop_2010_to_2019.csv", skiprows=3)
population_2020_2023 = pd.read_csv("csv_data_files/pop_2020_to_2023.csv", skiprows=3)

# Rename columns for clarity and clean state names
pop_2010_2019_cleaned = population_2010_2019.rename(columns={'Unnamed: 0': 'State'})
pop_2020_2023_cleaned = population_2020_2023.rename(columns={'Unnamed: 0': 'State'})

# Remove unnecessary columns
pop_2010_2019_cleaned = pop_2010_2019_cleaned.drop(columns=['April 1, 2010', 'Estimates Base'], errors='ignore')
pop_2020_2023_cleaned = pop_2020_2023_cleaned.drop(columns=['April 1, 2020 Estimates Base'], errors='ignore')

# Remove leading dots from state names
pop_2010_2019_cleaned['State'] = pop_2010_2019_cleaned['State'].str.strip('.')
pop_2020_2023_cleaned['State'] = pop_2020_2023_cleaned['State'].str.strip('.')

# Keep only rows for the selected states
selected_states = ['California', 'Florida', 'Texas', 'Washington', 'New Jersey']
pop_2010_2019_filtered = pop_2010_2019_cleaned[pop_2010_2019_cleaned['State'].isin(selected_states)]
pop_2020_2023_filtered = pop_2020_2023_cleaned[pop_2020_2023_cleaned['State'].isin(selected_states)]

# Remove commas from numeric columns and convert them to numeric
pop_2010_2019_filtered.iloc[:, 1:] = pop_2010_2019_filtered.iloc[:, 1:].replace({',': ''}, regex=True).apply(pd.to_numeric)
pop_2020_2023_filtered.iloc[:, 1:] = pop_2020_2023_filtered.iloc[:, 1:].replace({',': ''}, regex=True).apply(pd.to_numeric)

# Combine the datasets for 2016–2023
pop_2016_2019 = pop_2010_2019_filtered[['State', '2016', '2017', '2018', '2019']]
pop_2020_2023 = pop_2020_2023_filtered[['State', '2020', '2021', '2022', '2023']]
combined_population = pd.merge(pop_2016_2019, pop_2020_2023, on='State', how='outer')

# Melt the dataset to make it long format
melted_population = pd.melt(
    combined_population,
    id_vars=['State'],
    var_name='Year',
    value_name='Population'
)
melted_population['Year'] = melted_population['Year'].astype(int)
melted_population['Population'] = pd.to_numeric(melted_population['Population'], errors='coerce')

# Merge the population data with the registrations_data DataFrame
registrations_data = pd.merge(
    registrations_data,  # Replace with your actual DataFrame variable
    melted_population,
    on=['State', 'Year'],
    how='left'
)

# Add a new column for EV registrations per capita
registrations_data['EV_Per_Capita'] = registrations_data['Electric'] / registrations_data['Population']

print(registrations_data.head())



2. **Renewable and Total Energy Production by State**

**Cleaning Renewable Energy Data**

Filter on relevant years, states, and fuel types.


In [ ]:
clean_renewable_df = duckdb.sql("""
    SELECT Year, State, 
           SUM(CAST("Fuel Ethanol (T Btu)" AS FLOAT) + 
               CAST("Biodiesel (T Btu)" AS FLOAT) + 
               CAST("Renewable Diesel (T Btu)" AS FLOAT)) AS "Total Renewable (T Btu)"
    FROM thermal_units_df
    WHERE State IN ('California', 'Florida', 'Texas', 'Washington', 'New Jersey') AND Year > 2015
    GROUP BY Year, State
    ORDER BY Year, State
    """).df()

print(clean_renewable_df.head(n=10))

3. **EV Manufacturer Suggested Retail Prices and Annual Fuel Cost (by Models)**

**Cleaning EV Pricing Data**

To look into the pricing data, we clean the MSRP column from our pricing dataset, which contain values in different formats (e.g., ranges, with dollar signs, commas, etc.). We also drop rows with missing prices.

In [ ]:
# print(pricing_df_sorted['MSRP'])

# handle MSRP ranges and convert to numeric
def clean_msrp(msrp):
    if pd.isna(msrp) or msrp.strip() == '':  # Check for NaN or empty string
        return None
    msrp = msrp.replace('$', '').replace(',', '').strip()  # Remove $, commas, and spaces
    if '– ' in msrp:  # Handle price ranges
        low, high = msrp.split('–')
        return (float(low) + float(high)) / 2
    else:
        return float(msrp)  # Convert single price values to float

pricing_df_sorted['MSRP'] = pricing_df_sorted['MSRP'].apply(clean_msrp)

# drop rows where MSRP is NaN
pricing_df_sorted = pricing_df_sorted.dropna(subset=['MSRP'])

print(pricing_df_sorted)

**Create a Data Frame for the Lowest EV Prices across selected Years** 

We chose to focus on the lowest EV price of the year because it aligns with the cumulative nature of our other variables, such as the number of EV charging stations and incentives. The lowest price reflects the broader cumulative social and economic factors influencing EV affordability and accessibility, making it a meaningful indicator for our analysis. So here, we start calculating the lowest EV price for each year.

In [ ]:
# Get the lowest EV price of the year
lowest_prices_per_year = pricing_df_sorted.groupby('Year')['MSRP'].min().reset_index()
lowest_prices_per_year.rename(columns={'MSRP': 'Lowest_EV_Price'}, inplace=True)
print(lowest_prices_per_year.head())

4. **Electric Vehicle Charge Stations**

In [ ]:
ocm_us_df = pd.read_csv("csv_data_files/ocm_us_data.csv", dtype={"DateLastConfirmed": 'string'})

# converting DateCreated column to DateTime values
ocm_us_df["DateCreated"] = pd.to_datetime(ocm_us_df['DateCreated'], format='%m/%d/%Y %H:%M:%S %p')
ocm_us_df["YearCreated"] = ocm_us_df["DateCreated"].dt.year

# standardizing State column to state codes
state_map = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
state_map.update({v: v for v in state_map.values()})
ocm_us_df["StateOrProvince"] = ocm_us_df["StateOrProvince"].map(state_map)
ocm_us_df = ocm_us_df[ocm_us_df["StateOrProvince"].isin(state_map.keys())]
ocm_us_df["State"] = ocm_us_df["StateOrProvince"]


# grouping and counting stations by state, year
ocm_us_df_grouped = duckdb.sql("SELECT State, YearCreated AS Year, \
COUNT(State) AS NewStationsCreated FROM ocm_us_df GROUP BY State, \
YearCreated ORDER BY Year ASC, State ASC").df()
ocm_us_df_grouped["TotalStations"] = ocm_us_df_grouped.groupby("State")["NewStationsCreated"].cumsum()

# filtering for 2016 to 2023
ocm_us_df_limited = ocm_us_df_grouped.loc[(ocm_us_df_grouped["Year"] > 2015) \
& (ocm_us_df_grouped["Year"] < 2024)]

# combining with registered EV data
reg_data = pd.read_csv('csv_data_files/ev_registration_data.csv')
reg_data["State"] = reg_data["State"].map(state_map)
combined_df = duckdb.sql("SELECT reg_data.State, reg_data.Year, TotalStations, Electric \
FROM ocm_us_df_limited INNER JOIN reg_data ON reg_data.Year = ocm_us_df_limited.Year \
AND reg_data.State = ocm_us_df_limited.State ORDER BY reg_data.Year, reg_data.State").df()
combined_df["Electric"] = combined_df["Electric"].str.replace(",", "").astype(float)

# filtering for our 5 states
states_to_filter = ['CA', 'FL', 'TX', 'WA', 'NJ']
stations_df = combined_df[combined_df["State"].isin(states_to_filter)]

state_names = {
    "CA": "California",
    "FL": "Florida",
    "TX": "Texas",
    "WA": "Washington",
    "NJ": "New Jersey"
}

# Replace abbreviations with full names
stations_df.loc[:, "State"] = stations_df["State"].replace(state_names)

print(stations_df.head(n=10))
print(combined_df.shape)

4. **Electric Vehicle Federal Incentives** 

The federal incentives data cleaning process began with loading the raw data from a CSV file named 'state_laws_and_incentives.csv' into a pandas DataFrame. This initial dataset contained comprehensive information about various laws and incentives related to alternative fuels and vehicles across different jurisdictions. To make the data more focused for our analysis, we removed several columns that were not relevant. These included fields such as 'Law Id', 'Text', 'Recent?', and others related to agency details, dates of updates or repeals, and references. Removing these columns allowed us to concentrate on the key information needed to study federal incentives for electric vehicles.The next step addressed inconsistencies in the 'Enacted Date' and 'Expired Date' columns. These date fields were in mixed formats, which could lead to errors when analyzing time-based trends. Using pandas’ pd.to_datetime() function, we converted these columns into a consistent datetime format.

In [ ]:
# Load CSV file 
# (U.S. Department of Energy, Office of Energy Efficiency & Renewable Energy, 2024)
federal_incentives = pd.read_csv('./csv_data_files/state_laws_and_incentives.csv')

# Drop unecesary columns 
incentives_cols_to_drop = ['Law Id', 'Text', 'Recent?', 'Sequence Number', 
                           'Agency', 'Topic','Technology Categories', 'Archived Date',
                           'Amended Date', 'Repealed Date', 'User Categories',
                           'References', 'Status Date','Regulation Categories',
                           'Significant Update Date']

federal_incentives = federal_incentives.drop(columns=incentives_cols_to_drop)

# Convert each date column to the same datetime format
date_columns = ['Enacted Date', 'Expired Date']

for column in date_columns:
    federal_incentives[column] = pd.to_datetime(federal_incentives[column],
                                                 errors='coerce')
    # Extract the year directly
    federal_incentives[column] = federal_incentives[column].dt.year.astype('Int64')

# Check the resulting dataframe
print(federal_incentives)

To focus our analysis specifically on federal-level incentives, we applied a filter to the dataset. This step involved selecting only those rows where the 'State' column was labeled as 'US'. This filtering process ensured that our subsequent analyses would be based solely on nationwide policies and excluding state-specific incentives that could introduce variability into our study of federal trends.

In [ ]:
# Drop rows that contain state-level data
federal_incentives = federal_incentives[federal_incentives['State'] == 'US']

# Filter Type to only display federal incentives 
federal_incentives = federal_incentives[federal_incentives['Type'] == 'Incentives']

# Determine rows with <NA> enactment dates 
missing_federal_enacted_dates = federal_incentives[federal_incentives['Enacted Date'].isna()]
print(missing_federal_enacted_dates)

To fill gaps in the federal incentives data, missing information was identified and researched using reliable sources like government websites, policy documents, and industry reports. Verified data was manually added and corrected in the dataset, along with documentation of the sources. 

In [ ]:
# Manually enter enactment dates (where possible)

# Index 2 (Western Interstate Energy Board, n.d.)
federal_incentives.loc[2, 'Enacted Date'] = 1975

# Index 3 (U.S. Environmental Protection Agency, 2022)
federal_incentives.loc[3, 'Enacted Date'] = 2021

# Index 300 (Federal Highway Administration, n.d.)
federal_incentives.loc[300,'Enacted Date'] = 2015

# Index 325 (U.S. Environmental Protection Agency, 2024)
federal_incentives.loc[325, 'Enacted Date'] = 2024 

# Index 421 (WE ACT for Environmental Justice, 2023)
federal_incentives.loc[421, 'Enacted Date'] = 2023

# Index 422 (National Renewable Energy Laboratory, n.d.)
federal_incentives.loc[422, 'Enacted Date'] = 2018 

# Index 527 (Federal Highway Administration, 2023)
federal_incentives.loc[527, 'Enacted Date'] = 2023 

# Index 614 (U.S. Department of Transportation, n.d.)
federal_incentives.loc[614, 'Enacted Date'] = 2021 

# Index 643 (Joint Office of Energy and Transportation, 2023)
federal_incentives.loc[643, 'Enacted Date'] = 2023 

# Index 668 (U.S. Department of Energy, 2023b)
federal_incentives.loc[668, 'Enacted Date'] = 2023 

# Indices 670-675 (U.S. Department of Agriculture, 2022)
indices = [670, 671, 672, 673, 674, 675]
federal_incentives.loc[indices, 'Enacted Date'] = 2022 

Sorting the dataset by enacted date helps show the timeline of electric vehicle (EV) incentives, making it easier to see when different policies were introduced. This allows us to identify patterns in policy changes and how they relate to EV registrations.

In [ ]:
# Sort by enacted date 
federal_incentives = federal_incentives.sort_values(by='Enacted Date', 
                                                            ascending=True)
federal_incentives.reset_index(drop=True, inplace=True)

print(federal_incentives.head(10))

Here, we are processing data on electric vehicle (EV) registrations and federal incentives to create a dataset for analysis. It calculates the number of active federal incentives for each year from 2016 to 2023, considering incentives as active if enacted before or during the year and not expired. We then filter EV registration data for five key states — California, Florida, Texas, Washington, and New Jersey. Finally, it combines the state-level registration data with federal incentive counts to create a dataset that will help us to analyze how EV adoption in these states correlates with federal incentives.

In [ ]:
# Prepare list of years
years = list(range(2016, 2024))

# Count active incentives for each year
incentives_per_year = []
for year in years:
    active_incentives = federal_incentives[
        (federal_incentives['Enacted Date'] <= year) & 
        ((federal_incentives['Expired Date'] > year) | 
         (federal_incentives['Expired Date'].isna()))
    ]
    incentives_per_year.append(len(active_incentives))

# Create federal incentives dataframe
federal_incentives = pd.DataFrame({
    'Year': years,
    'Federal Incentives Count': incentives_per_year
})

# States of interest
states_of_interest = ['California', 'Florida', 'Texas', 'Washington', 'New Jersey']

# Create final dataframe with EV registrations and incentives
final_incentives_df = (
    compiled_df[compiled_df['State'].isin(states_of_interest)]
    .groupby(['State','Year'])['Electric']
    .sum()
    .reset_index()
    .merge(federal_incentives, on='Year', how='left')
)

print(final_incentives_df)

## Data Cleaning - Merging and Integrating the Dataset:

**Merging the Data Frames into One Dataset** 

To create a comprehensive dataset for analysis, we merged the EV pricing data with the registration data using the Year column as the key. 

In [ ]:
# Merge the filtered registration data with other data

## Merge with Price Data (lowest price by year)
merged_df = pd.merge(
    registrations_data, 
    lowest_prices_per_year, 
    on='Year', 
    how='left')

# Merge with Federal Incentives Data (only merge specific columns)
merged_df = pd.merge(
    merged_df,
    final_incentives_df[['Year', 'State', 'Federal Incentives Count']],
    on=['Year', 'State'],
    how='left'
)

## Merge Renewable Energy Data
merged_df = pd.merge(
    merged_df,
    clean_renewable_df,
    on=['Year', 'State'],
    how='left'
)

## Merge Charging Stations Data
merged_df = pd.merge(
    merged_df,
    stations_df[['Year', 'State', 'TotalStations']],
    on=['Year', 'State'],
    how='left'
)

print(merged_df.head(12))

In [ ]:
# Rename columns
merged_df.rename(
    columns={
        "Electric": "registration", 
        "EV_Per_Capita": "reg_per_capita",
        "Lowest_EV_Price": "min_price", 
        "Federal Incentives Count": "incentives", 
        "Total Renewable (T Btu)": "renewable_prod", 
        "TotalStations": "charge_stations",
    }, 
    inplace=True
)

merged_df.columns = merged_df.columns.str.lower()

merged_df["charge_per_capita"] = merged_df["charge_stations"] / merged_df["population"] * 100000

print(merged_df.head(12))

**Saving the final dataset to a CSV** 

For future analysis, we saved the dataset to a csv.

In [ ]:
#Save the final dataset to a CSV for further analysis (uncomment and call this after all data cleaning and merging are done)
# merged_df.to_csv('final_ev_dataset.csv', index=False)

## Ackowledgements and Bibliography

<p style="padding-left: 40px; text-indent: -40px;">
    Federal Highway Administration. (2023, March 14). <i>Biden-Harris Administration opens applications for first round of $2.5 billion program to build EV charging infrastructure</i>. Retrieved from <a href="https://highways.dot.gov/newsroom/biden-harris-administration-opens-applications-first-round-25-billion-program-build-ev">https://highways.dot.gov/newsroom/biden-harris-administration-opens-applications-first-round-25-billion-program-build-ev</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    Federal Highway Administration. (n.d.). <i>Fixing America's Surface Transportation Act or "FAST Act"</i>. Retrieved from <a href="https://ops.fhwa.dot.gov/fastact/#:~:text=Fixing%20America's%20Surface%20Transportation%20Act%20or%20%22FAST%20Act%22&text=On%20December%204%2C%202015%2C%20President,funding%20certainty%20for%20surface%20transportation.">https://ops.fhwa.dot.gov/fastact/#:~:text=Fixing%20America's%20Surface%20Transportation%20Act%20or%20"FAST Act"</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    Joint Office of Energy and Transportation (2023, September 21). <i>EV Charger Reliability and Accessibility Accelerator Webinar</i>. Retrieved from <a href="https://driveelectric.gov/webinars/ev-charger-reliability-accessibility-accelerator">https://driveelectric.gov/webinars/ev-charger-reliability-accessibility-accelerator</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    National Renewable Energy Laboratory. (n.d.). <i>American Made Program 5-Year Report</i>. Retrieved from <a href="https://americanmadechallenges.org/American-Made-Program-5-Year-Report.pdf">https://americanmadechallenges.org/American-Made-Program-5-Year-Report.pdf</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    Open Charge Map. (2024). <i>Open Charge Map API</i>. Retrieved November 20, 2024, from <a href="https://openchargemap.org/site/develop/api">https://openchargemap.org/site/develop/api</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
U.S. Census Bureau, Population Division. (2019). Annual Estimates of the Resident Population for the United States, Regions, States, and Puerto Rico: April 1, 2010 to July 1, 2019 (NST-EST2019-01) [Data file]. Retrieved from <a href="https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx">https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Census Bureau, Population Division. (2023). <i>Annual Estimates of the Resident Population for the United States, Regions, States, District of Columbia, and Puerto Rico: April 1, 2020 to July 1, 2023 (NST-EST2023-POP) [Data file]</i>. Retrieved from <a href="https://www2.census.gov/programs-surveys/popest/tables/2020-2023/state/totals/NST-EST2023-POP.xlsx">https://www2.census.gov/programs-surveys/popest/tables/2020-2023/state/totals/NST-EST2023-POP.xlsx</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Agriculture. (2022). <i>EV Charging Stations Fact Sheet</i>. Retrieved from <a href="https://www.rd.usda.gov/sites/default/files/508_RD_FS_71EVChargingStations.pdf">https://www.rd.usda.gov/sites/default/files/508_RD_FS_71EVChargingStations.pdf</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Energy. (2023a). <i>Alternative Fuels Data Center: Vehicle Registration Data</i>. Retrieved from <a href="https://afdc.energy.gov/vehicle-registration?year=2023">https://afdc.energy.gov/vehicle-registration?year=2023</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Energy. (2023b). <i>Domestic Manufacturing Conversion Grants</i>. Retrieved from <a href="https://www.energy.gov/mesc/domestic-manufacturing-conversion-grants">https://www.energy.gov/mesc/domestic-manufacturing-conversion-grants</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Energy. (n.d.). <i>Find and Compare Cars</i>. Retrieved from <a href="https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023&minmsrpsel=0&maxmsrpsel=0&city=0&hwy=0&comb=0&cbvtelectric=Electric&YearSel=2016-2023&make=&mclass=&vfuel=&vtype=Electric&trany=&drive=&cyl=&MpgSel=000&sortBy=Comb&Units=&url=SearchServlet&opt=new&minmsrp=0&maxmsrp=0&minmpg=0&maxmpg=0&sCharge=&tCharge=&startstop=&cylDeact=&rowLimit=200&pageno=1&tabView=0">https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&year1=2016&year2=2023</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Energy, Office of Energy Efficiency & Renewable Energy. (2024). <i>Alternative Fuels Data Center: Data Download [Data file]</i>. Retrieved November 20, 2024, from <a href="https://afdc.energy.gov/data_download">https://afdc.energy.gov/data_download</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Department of Transportation. (n.d.). <i>Resilient Surface Transportation Grants</i>. Retrieved from <a href="https://www.transit.dot.gov/grants?combine=Resilient+Surface+Transportation+Grants&field_grant_type_target_id=All">https://www.transit.dot.gov/grants?combine=Resilient+Surface+Transportation+Grants&field_grant_type_target_id=All</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Energy Information Administration. (n.d.). <i>State Energy Data System (SEDS): 1960-2021 (Complete) [Data file]</i>. Retrieved from <a href="https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf">https://www.eia.gov/state/seds/sep_prod/SEDS_Production_Report.pdf</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Environmental Protection Agency. (2022). <i>Bipartisan Infrastructure Law Clean School Bus Program: Initial Implementation Report to Congress</i>. Retrieved from <a href="https://nepis.epa.gov/Exe/ZyPDF.cgi?Dockey=P1014098.pdf">https://nepis.epa.gov/Exe/ZyPDF.cgi?Dockey=P1014098.pdf</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    U.S. Environmental Protection Agency. (2024, April 24). <i>Biden-Harris Administration announces nearly $1 billion in grants to invest in America's clean energy future</i>. Retrieved from <a href="https://www.epa.gov/newsreleases/biden-harris-administration-announces-nearly-1-billion-grants-invest-americas-clean-0">https://www.epa.gov/newsreleases/biden-harris-administration-announces-nearly-1-billion-grants-invest-americas-clean-0</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    WE ACT for Environmental Justice. (2023). <i>TCTAC</i>. Retrieved from <a href="https://www.weact.org/tctac/">https://www.weact.org/tctac/</a>
</p>

<p style="padding-left: 40px; text-indent: -40px;">
    Western Interstate Energy Board. (n.d.). <i>Western Interstate Energy Board supports continued funding for the U.S. Department of Energy State Energy Program (SEP)</i>. Retrieved from <a href="https://www.westernenergyboard.org/western-interstate-energy-board-supports-continued-funding-for-the-u-s-department-of-energy-state-energy-program-sep/">https://www.westernenergyboard.org/western-interstate-energy-board-supports-continued-funding-for-the-u-s-department-of-energy-state-energy-program-sep/</a>
</p>
